# Library

In [1]:
import numpy as np, os
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# df_all

In [2]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

In [3]:
# 2) MODEL
from keras.models import load_model
lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

Using TensorFlow backend.


## 바로가기

In [6]:
# 3)
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_mean_lst = []
results = []
preds = lstm2.predict(X_test)
preds_mean = np.mean(preds)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()
    X_test[:,:,k] = 1
#     X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
#     X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
#     X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

    # COMPUTE BCE WITH FEATURE K REVERSED, preds2_mean_list
    preds2 = lstm2.predict(X_test)
#     preds2_mean = np.mean(preds2)
    preds2_mean_lst.append(np.mean(preds2))
    
    loss_bce = bce(y_test, preds2).numpy()
    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [33:16<00:00,  2.04it/s]


In [9]:
# results
# preds2_mean_lst

[{'feature': 'BASELINE', 'baseline_bce': 1.0138444},
 {'feature': 0, 'bce': 1.012852},
 {'feature': 50803, 'bce': 1.0162216},
 {'feature': 50804, 'bce': 1.0177649},
 {'feature': 50805, 'bce': 1.015766},
 {'feature': 50806, 'bce': 1.0188677},
 {'feature': 50808, 'bce': 1.0160338},
 {'feature': 50809, 'bce': 1.0169848},
 {'feature': 50811, 'bce': 1.0182089},
 {'feature': 50813, 'bce': 1.0262372},
 {'feature': 50814, 'bce': 1.0160587},
 {'feature': 50818, 'bce': 1.0162146},
 {'feature': 50820, 'bce': 1.0172079},
 {'feature': 50821, 'bce': 1.0159402},
 {'feature': 50822, 'bce': 1.0150534},
 {'feature': 50824, 'bce': 1.0172732},
 {'feature': 50852, 'bce': 1.0136635},
 {'feature': 50853, 'bce': 1.0167024},
 {'feature': 50856, 'bce': 1.026412},
 {'feature': 50858, 'bce': 1.0135549},
 {'feature': 50861, 'bce': 1.0263983},
 {'feature': 50862, 'bce': 1.0230782},
 {'feature': 50863, 'bce': 1.0155033},
 {'feature': 50864, 'bce': 1.0150094},
 {'feature': 50865, 'bce': 1.0148703},
 {'feature': 50866

[0.6136082,
 0.607474,
 0.6663015,
 0.59963495,
 0.6821946,
 0.64452255,
 0.6637433,
 0.67241734,
 0.71189034,
 0.5961122,
 0.644227,
 0.6598693,
 0.6479512,
 0.6379032,
 0.6654543,
 0.61547756,
 0.5866017,
 0.53938377,
 0.6295676,
 0.54032063,
 0.7084942,
 0.62790036,
 0.6524332,
 0.6030097,
 0.6190605,
 0.5552965,
 0.71592087,
 0.6322274,
 0.6337536,
 0.6184313,
 0.6192685,
 0.6319263,
 0.60304207,
 0.6058177,
 0.69250685,
 0.6158326,
 0.5979262,
 0.51824915,
 0.5723266,
 0.58894295,
 0.6319717,
 0.6623113,
 0.6304232,
 0.64744234,
 0.6510889,
 0.6526949,
 0.57916933,
 0.67024875,
 0.6560272,
 0.6361712,
 0.6208487,
 0.6335528,
 0.7060838,
 0.68175757,
 0.60723394,
 0.68571895,
 0.6880517,
 0.63149697,
 0.6100433,
 0.5852619,
 0.6117147,
 0.6964605,
 0.6354073,
 0.5663321,
 0.55905265,
 0.6350505,
 0.63497376,
 0.6676746,
 0.6356122,
 0.56102693,
 0.61912954,
 0.6355207,
 0.61953366,
 0.6351556,
 0.61155367,
 0.6019964,
 0.63172317,
 0.63625544,
 0.5727521,
 0.5568934,
 0.63168925,
 

## 바로가기2

In [17]:
df1 = pd.DataFrame(results)
df1

,feature,baseline_bce,bce
0,BASELINE,1.013844,NaN
1,0,NaN,1.012852
2,50803,NaN,1.016222
3,50804,NaN,1.017765
4,50805,NaN,1.015766
...,...,...,...
4064,78112001103,NaN,1.016546
4065,79511050204,NaN,1.013754
4066,87701071218,NaN,1.026336
4067,87701083336,NaN,1.013647


In [18]:
# len(preds2_mean_lst)

4069

In [32]:
#list 특정 위치에 추가는 list.insert(인덱스, 추가할 값)
# preds2_mean_lst.insert(0, np.NaN)
preds2_mean_lst
# del preds2_mean_lst[0]

[nan,
 0.6136082,
 0.607474,
 0.6663015,
 0.59963495,
 0.6821946,
 0.64452255,
 0.6637433,
 0.67241734,
 0.71189034,
 0.5961122,
 0.644227,
 0.6598693,
 0.6479512,
 0.6379032,
 0.6654543,
 0.61547756,
 0.5866017,
 0.53938377,
 0.6295676,
 0.54032063,
 0.7084942,
 0.62790036,
 0.6524332,
 0.6030097,
 0.6190605,
 0.5552965,
 0.71592087,
 0.6322274,
 0.6337536,
 0.6184313,
 0.6192685,
 0.6319263,
 0.60304207,
 0.6058177,
 0.69250685,
 0.6158326,
 0.5979262,
 0.51824915,
 0.5723266,
 0.58894295,
 0.6319717,
 0.6623113,
 0.6304232,
 0.64744234,
 0.6510889,
 0.6526949,
 0.57916933,
 0.67024875,
 0.6560272,
 0.6361712,
 0.6208487,
 0.6335528,
 0.7060838,
 0.68175757,
 0.60723394,
 0.68571895,
 0.6880517,
 0.63149697,
 0.6100433,
 0.5852619,
 0.6117147,
 0.6964605,
 0.6354073,
 0.5663321,
 0.55905265,
 0.6350505,
 0.63497376,
 0.6676746,
 0.6356122,
 0.56102693,
 0.61912954,
 0.6355207,
 0.61953366,
 0.6351556,
 0.61155367,
 0.6019964,
 0.63172317,
 0.63625544,
 0.5727521,
 0.5568934,
 0.63168

In [20]:
# preds_mean

0.6335612

In [24]:
preds_mean_lst = []
for i in range(len(df1)):
    if i == 0:
        preds_mean_lst.append(preds_mean)
    else:
        preds_mean_lst.append(np.nan)

len(preds_mean_lst)
preds_mean_lst

4069

[0.6335612,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [26]:
# (np.NaN)*(len(df1)-1) # 이렇게 한다고 np.nan 복제되지 않음.
# preds_mean_lst

In [27]:
df1['preds_mean'] = preds_mean_lst

In [33]:
df1['preds2_mean'] = preds2_mean_lst
df1

,feature,baseline_bce,bce,preds_mean,preds2_mean
0,BASELINE,1.013844,NaN,0.633561,NaN
1,0,NaN,1.012852,NaN,0.613608
2,50803,NaN,1.016222,NaN,0.607474
3,50804,NaN,1.017765,NaN,0.666301
4,50805,NaN,1.015766,NaN,0.599635
...,...,...,...,...,...
4064,78112001103,NaN,1.016546,NaN,0.583690
4065,79511050204,NaN,1.013754,NaN,0.631427
4066,87701071218,NaN,1.026336,NaN,0.722150
4067,87701083336,NaN,1.013647,NaN,0.640804


In [35]:
df1.iloc[0,3]

0.6335611939430237

## 바로가기3

In [37]:
df_all_best = df1.copy()

base_value = df1.iloc[0,1]
df_all_best['baseline_bce'] = base_value
df_all_best
df_all_best['diff'] = df_all_best['bce']-df_all_best['baseline_bce']
# df_all_best

,feature,baseline_bce,bce,preds_mean,preds2_mean
0,BASELINE,1.013844,NaN,0.633561,NaN
1,0,1.013844,1.012852,NaN,0.613608
2,50803,1.013844,1.016222,NaN,0.607474
3,50804,1.013844,1.017765,NaN,0.666301
4,50805,1.013844,1.015766,NaN,0.599635
...,...,...,...,...,...
4064,78112001103,1.013844,1.016546,NaN,0.583690
4065,79511050204,1.013844,1.013754,NaN,0.631427
4066,87701071218,1.013844,1.026336,NaN,0.722150
4067,87701083336,1.013844,1.013647,NaN,0.640804


In [38]:
base_preds = df1.iloc[0,3]
df_all_best['preds_mean'] = base_preds
df_all_best['diff_preds'] = df_all_best['preds2_mean'] - df_all_best['preds_mean']
df_all_best

,feature,baseline_bce,bce,preds_mean,preds2_mean,diff,diff_preds
0,BASELINE,1.013844,NaN,0.633561,NaN,NaN,NaN
1,0,1.013844,1.012852,0.633561,0.613608,-0.000992,-0.019953
2,50803,1.013844,1.016222,0.633561,0.607474,0.002377,-0.026087
3,50804,1.013844,1.017765,0.633561,0.666301,0.003921,0.032740
4,50805,1.013844,1.015766,0.633561,0.599635,0.001922,-0.033926
...,...,...,...,...,...,...,...
4064,78112001103,1.013844,1.016546,0.633561,0.583690,0.002702,-0.049871
4065,79511050204,1.013844,1.013754,0.633561,0.631427,-0.000091,-0.002134
4066,87701071218,1.013844,1.026336,0.633561,0.722150,0.012491,0.088589
4067,87701083336,1.013844,1.013647,0.633561,0.640804,-0.000197,0.007243


## 바로가기4

In [39]:
df_all_best = df_all_best.drop(index=[0])

In [40]:
df_all_best = df_all_best.sort_values(by=['diff', 'diff_preds'])
df_all_best

,feature,baseline_bce,bce,preds_mean,preds2_mean,diff,diff_preds
2612,904224461,1.013844,1.011739,0.633561,0.622529,-0.002106,-0.011032
1092,69152068,1.013844,1.012304,0.633561,0.633442,-0.001540,-0.000120
2338,536242558,1.013844,1.012476,0.633561,0.639803,-0.001368,0.006242
1525,91464024,1.013844,1.012594,0.633561,0.640399,-0.001250,0.006838
2878,37205036174,1.013844,1.012601,0.633561,0.639982,-0.001243,0.006421
...,...,...,...,...,...,...,...
2745,10019055302,1.013844,1.041340,0.633561,0.766674,0.027495,0.133112
983,54817525,1.013844,1.042170,0.633561,0.767467,0.028326,0.133906
1082,67434504,1.013844,1.046685,0.633561,0.777882,0.032840,0.144320
1277,74606211,1.013844,1.050331,0.633561,0.784815,0.036486,0.151254


In [47]:
diffdf = df_all_best[['feature','diff']]
diffdf = diffdf.sort_values(by='diff')
diffdf

,feature,diff
2612,904224461,-0.002106
1092,69152068,-0.001540
2338,536242558,-0.001368
1525,91464024,-0.001250
2878,37205036174,-0.001243
...,...,...
2745,10019055302,0.027495
983,54817525,0.028326
1082,67434504,0.032840
1277,74606211,0.036486


In [49]:
diffpreds_df = df_all_best[['feature','diff_preds']]
diffpreds_df = diffpreds_df.sort_values(by='diff_preds')
diffpreds_df

,feature,diff_preds
384,227194,-0.136509
470,4003822,-0.122565
38,50889,-0.115312
3819,63323016101,-0.114032
163,51200,-0.114011
...,...,...
2745,10019055302,0.133112
983,54817525,0.133906
1082,67434504,0.144320
1277,74606211,0.151254


In [46]:
# df_all_best.to_csv('df_all_best_for_meth1.csv',index = False)

In [51]:
# diffdf.to_csv('df_all_diff_for_meth1.csv',index = False)
# diffpreds_df.to_csv('df_all_preds_for_meth1.csv',index = False)

# df_1to5

In [53]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

In [54]:
# 2) MODEL
from keras.models import load_model
lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

## 바로가기

In [55]:
# 3)
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_mean_lst = []
results = []
preds = lstm2.predict(X_test)
preds_mean = np.mean(preds)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()
    X_test[:,5:,k] = 1
#     X_test[:,5:,k] = np.where(X_test[:,5:,k]==1, 2, X_test[:,5:,k])
#     X_test[:,5:,k] = np.where(X_test[:,5:,k]==0, 1, X_test[:,5:,k])
#     X_test[:,5:,k] = np.where(X_test[:,5:,k]==2, 0, X_test[:,5:,k])

    # COMPUTE BCE WITH FEATURE K REVERSED, , preds2_mean_list
    preds2 = lstm2.predict(X_test)
    preds2_mean_lst.append(np.mean(preds2))
    
    loss_bce = bce(y_test, preds).numpy()
    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [33:16<00:00,  2.04it/s]


## 바로가기2

In [56]:
# 4) DISPLAY LSTM FEATURE IMPORTANCE
df3 = pd.DataFrame(results)

################################################################################
preds2_mean_lst.insert(0, np.NaN)
# preds2_mean_lst

preds_mean_lst = []
for i in range(len(df3)):
    if i == 0:
        preds_mean_lst.append(preds_mean)
    else:
        preds_mean_lst.append(np.nan)

# len(preds_mean_lst)
# preds_mean_lst
df3['preds_mean'] = preds_mean_lst
df3['preds2_mean'] = preds2_mean_lst
################################################################################

df_d1d5_best = df3.copy()

base_value = df3.iloc[0,1]
df_d1d5_best['baseline_bce'] = base_value
df_d1d5_best['diff'] = df_d1d5_best['bce']-df_d1d5_best['baseline_bce']

#############################################################
base_preds = df3.iloc[0,3]
df_d1d5_best['preds_mean'] = base_preds
df_d1d5_best['diff_preds'] = df_d1d5_best['preds2_mean'] - df_d1d5_best['preds_mean']
# df_d1d5_best
#############################################################

df_d1d5_best = df_d1d5_best.drop(index=[0])

################################################################################
df_d1d5_best = df_d1d5_best.sort_values(by=['diff', 'diff_preds'])
df_d1d5_best

diffdf = df_d1d5_best[['feature','diff']]
diffdf = diffdf.sort_values(by='diff')
diffdf

diffpreds_df = df_d1d5_best[['feature','diff_preds']]
diffpreds_df = diffpreds_df.sort_values(by='diff_preds')
diffpreds_df
################################################################################

[nan,
 0.6242791,
 0.62211585,
 0.67211163,
 0.61032355,
 0.67960644,
 0.6539321,
 0.66951835,
 0.67451715,
 0.70562154,
 0.6074597,
 0.654519,
 0.66709614,
 0.6565505,
 0.6465491,
 0.667332,
 0.6211919,
 0.59910005,
 0.56403077,
 0.63352275,
 0.56729263,
 0.69735885,
 0.6363129,
 0.65345114,
 0.61307466,
 0.6265191,
 0.577246,
 0.7068301,
 0.63582903,
 0.6372185,
 0.6257308,
 0.62521553,
 0.6356602,
 0.61808586,
 0.61523205,
 0.6849411,
 0.62439746,
 0.612122,
 0.5467979,
 0.59012103,
 0.60512674,
 0.63892186,
 0.65932256,
 0.634412,
 0.64776385,
 0.6515293,
 0.6517613,
 0.59543467,
 0.6657598,
 0.65550125,
 0.64009506,
 0.62685525,
 0.63452387,
 0.6967271,
 0.6783794,
 0.6203949,
 0.6822549,
 0.6795181,
 0.6351694,
 0.6171202,
 0.5985362,
 0.61891305,
 0.6890497,
 0.63876015,
 0.58613384,
 0.5744354,
 0.6381411,
 0.6380826,
 0.6640913,
 0.63843447,
 0.5819109,
 0.6268036,
 0.63936543,
 0.6288286,
 0.6384697,
 0.61929005,
 0.61238706,
 0.6352112,
 0.63797736,
 0.59061164,
 0.5779371,


,feature,baseline_bce,bce,preds_mean,preds2_mean,diff,diff_preds
0,BASELINE,1.014362,NaN,0.636933,NaN,NaN,NaN
1,0,1.014362,1.014362,0.636933,0.624279,0.0,-0.012654
2,50803,1.014362,1.014362,0.636933,0.622116,0.0,-0.014817
3,50804,1.014362,1.014362,0.636933,0.672112,0.0,0.035179
4,50805,1.014362,1.014362,0.636933,0.610324,0.0,-0.026609
...,...,...,...,...,...,...,...
4064,78112001103,1.014362,1.014362,0.636933,0.595469,0.0,-0.041464
4065,79511050204,1.014362,1.014362,0.636933,0.635178,0.0,-0.001754
4066,87701071218,1.014362,1.014362,0.636933,0.704860,0.0,0.067927
4067,87701083336,1.014362,1.014362,0.636933,0.641592,0.0,0.004660


,feature,baseline_bce,bce,preds_mean,preds2_mean,diff,diff_preds
384,227194,1.014362,1.014362,0.636933,0.531525,0.0,-0.105408
470,4003822,1.014362,1.014362,0.636933,0.539771,0.0,-0.097162
3819,63323016101,1.014362,1.014362,0.636933,0.544948,0.0,-0.091985
163,51200,1.014362,1.014362,0.636933,0.546589,0.0,-0.090344
38,50889,1.014362,1.014362,0.636933,0.546798,0.0,-0.090135
...,...,...,...,...,...,...,...
983,54817525,1.014362,1.014362,0.636933,0.746177,0.0,0.109244
2745,10019055302,1.014362,1.014362,0.636933,0.747865,0.0,0.110932
1082,67434504,1.014362,1.014362,0.636933,0.758072,0.0,0.121139
1277,74606211,1.014362,1.014362,0.636933,0.764385,0.0,0.127452


,feature,diff
384,227194,0.0
2359,548590000,0.0
1727,173011318,0.0
3844,63323031461,0.0
3244,51079089920,0.0
...,...,...
3927,64894082550,0.0
3864,63323054011,0.0
208,51279,0.0
2747,10019063101,0.0


,feature,diff_preds
384,227194,-0.105408
470,4003822,-0.097162
3819,63323016101,-0.091985
163,51200,-0.090344
38,50889,-0.090135
...,...,...
983,54817525,0.109244
2745,10019055302,0.110932
1082,67434504,0.121139
1277,74606211,0.127452


In [57]:
df_d1d5_best.to_csv('df_d1d5_best_for_meth1.csv',index = False)

diffdf.to_csv('df_d1d5_diff_for_meth1.csv',index = False)
diffpreds_df.to_csv('df_d1d5_preds_for_meth1.csv',index = False)

# df_6to10

In [58]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

In [59]:
# 2) MODEL
from keras.models import load_model
lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

## 바로가기

In [60]:
# 3)
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_mean_lst = []
results = []
preds = lstm2.predict(X_test)
preds_mean = np.mean(preds)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()
    X_test[:,0:5,k] = 1
#     X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==1, 2, X_test[:,0:5,k])
#     X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==0, 1, X_test[:,0:5,k])
#     X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==2, 0, X_test[:,0:5,k])

    # COMPUTE BCE WITH FEATURE K REVERSED, preds2_mean_list
    preds2 = lstm2.predict(X_test)
    preds2_mean_lst.append(np.mean(preds2))
    
    loss_bce = bce(y_test, preds).numpy()
    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [33:23<00:00,  2.03it/s]


In [ ]:
# 경로 /project/LSH/

# 3)에서 추가해야할 부분들, 그리고 for문 안 preds 이름 preds2로 바꿔줘야함.

# preds2_mean_lst = []
# preds_mean = np.mean(preds)
# # , preds2_mean_list
# preds2_mean_lst.append(np.mean(preds2))

## 바로가기2

In [61]:
# 4) DISPLAY LSTM FEATURE IMPORTANCE
df3 = pd.DataFrame(results)

################################################################################
preds2_mean_lst.insert(0, np.NaN)
# preds2_mean_lst

preds_mean_lst = []
for i in range(len(df3)):
    if i == 0:
        preds_mean_lst.append(preds_mean)
    else:
        preds_mean_lst.append(np.nan)

# len(preds_mean_lst)
# preds_mean_lst
df3['preds_mean'] = preds_mean_lst
df3['preds2_mean'] = preds2_mean_lst
################################################################################

df_d6d10_best = df3.copy()

base_value = df3.iloc[0,1]
df_d6d10_best['baseline_bce'] = base_value
df_d6d10_best['diff'] = df_d6d10_best['bce']-df_d6d10_best['baseline_bce']

#############################################################
base_preds = df3.iloc[0,3]
df_d6d10_best['preds_mean'] = base_preds
df_d6d10_best['diff_preds'] = df_d6d10_best['preds2_mean'] - df_d6d10_best['preds_mean']
# df_d6d10_best
#############################################################

df_d6d10_best = df_d6d10_best.drop(index=[0])

################################################################################
df_d6d10_best = df_d6d10_best.sort_values(by=['diff', 'diff_preds'])
df_d6d10_best

diffdf = df_d6d10_best[['feature','diff']]
diffdf = diffdf.sort_values(by='diff')
diffdf

diffpreds_df = df_d6d10_best[['feature','diff_preds']]
diffpreds_df = diffpreds_df.sort_values(by='diff_preds')
diffpreds_df
################################################################################

,feature,baseline_bce,bce,preds_mean,preds2_mean,diff,diff_preds
384,227194,1.014362,1.014362,0.636933,0.605696,0.0,-0.031237
470,4003822,1.014362,1.014362,0.636933,0.611667,0.0,-0.025266
38,50889,1.014362,1.014362,0.636933,0.612010,0.0,-0.024923
163,51200,1.014362,1.014362,0.636933,0.613429,0.0,-0.023503
1816,182138167,1.014362,1.014362,0.636933,0.613611,0.0,-0.023322
...,...,...,...,...,...,...,...
2745,10019055302,1.014362,1.014362,0.636933,0.664298,0.0,0.027365
1082,67434504,1.014362,1.014362,0.636933,0.666336,0.0,0.029404
983,54817525,1.014362,1.014362,0.636933,0.667255,0.0,0.030322
1277,74606211,1.014362,1.014362,0.636933,0.667697,0.0,0.030764


,feature,diff
384,227194,0.0
682,9361703,0.0
204,51274,0.0
2153,409190301,0.0
3959,66479024725,0.0
...,...,...
3776,62584073301,0.0
2770,11098001001,0.0
1561,93738001,0.0
3530,57706075560,0.0


,feature,diff_preds
384,227194,-0.031237
470,4003822,-0.025266
38,50889,-0.024923
163,51200,-0.023503
1816,182138167,-0.023322
...,...,...
2745,10019055302,0.027365
1082,67434504,0.029404
983,54817525,0.030322
1277,74606211,0.030764


In [62]:
df_d6d10_best.to_csv('df_d6d10_best_for_meth1.csv',index = False)

diffdf.to_csv('df_d6d10_diff_for_meth1.csv',index = False)
diffpreds_df.to_csv('df_d6d10_preds_for_meth1.csv',index = False)